In [1]:
import pyspark

In [2]:
#from pyspark.sql import SQLContext
sc = pyspark.SparkContext()

In [3]:
from pyspark.sql import *

In [4]:
spark = SparkSession.builder.appName("DIC assignment").config("spark.some.config.option","some-value").getOrCreate()

In [5]:
df2 = spark.read.csv('train.csv',inferSchema =True, header= True, escape="\"")


In [6]:
df3 = spark.read.csv('test.csv',inferSchema =True, header= True, escape="\"")

In [8]:
test_df = df3.dropna(thresh=3,subset=('movie_id','movie_name','plot'))

In [9]:
df = df2.dropna(thresh=4,subset=('movie_id','movie_name','plot','genre'))

In [ ]:
def convertBinary(z):
    import ast
    import re
    init_list =[0 for i in range(20)] 
    string = ""
    genre = ["Drama", "Comedy", "Romance Film", "Thriller", "Action", "World cinema", "Crime Fiction", "Horror", "Black-and-white", "Indie","Action/Adventure", "Adventure", "Family Film", "Short Film", "Romantic drama","Animation","Musical","Science Fiction","Mystery","Romantic comedy"]
    temp = ast.literal_eval(z)
    z = temp   
    
    for i in range(len(z)): 
        if z[i] in genre:
            index = genre.index(z[i])
            init_list[index] = 1  

    
    for j in init_list:
        string += str(j)
        string += ' '
    return string

In [17]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,IntegerType
colsInt = udf(lambda z: convertBinary(z), StringType())
spark.udf.register("colsInt", colsInt)
df2 = df.withColumn( 'binary',colsInt('genre'))
df2.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|              binary|
+--------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|1 0 0 0 0 1 0 0 0...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|1 0 0 0 1 0 0 0 0...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|1 0 0 0 1 0 0 0 0...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|0 1 0 0 0 0 0 0 0...|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|1 0 0 0 0 1 1 0 0...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|1 0 0 1 1 0 0 0 0...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|1 0 0 1 0 0 0 1 0...|
|24225279|                Sing|The story begins ..

In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, Word2Vec
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
tokenized = regexTokenizer.transform(df2)

# stop words
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
stop = stopwordsRemover.transform(tokenized)

# bag of words count
word2vec = Word2Vec(vectorSize = 100,minCount = 5, inputCol = 'filtered', outputCol = 'features')
model = word2vec.fit(stop)
result = model.transform(stop)

split_col = pyspark.sql.functions.split(result['binary'], ' ')
result = result.withColumn('Drama', split_col.getItem(0).cast(IntegerType()))
result = result.withColumn('Comedy', split_col.getItem(1).cast(IntegerType()))
result = result.withColumn('Romance Film', split_col.getItem(2).cast(IntegerType()))
result = result.withColumn('Thriller', split_col.getItem(3).cast(IntegerType()))
result = result.withColumn('Action', split_col.getItem(4).cast(IntegerType()))
result = result.withColumn('World Cinema', split_col.getItem(5).cast(IntegerType()))
result = result.withColumn('Crime Fiction', split_col.getItem(6).cast(IntegerType()))
result = result.withColumn('Horror', split_col.getItem(7).cast(IntegerType()))
result = result.withColumn('Black-and-White', split_col.getItem(8).cast(IntegerType()))
result = result.withColumn('Indie', split_col.getItem(9).cast(IntegerType()))
result = result.withColumn('Action/Adventure', split_col.getItem(10).cast(IntegerType()))
result = result.withColumn('Adventure', split_col.getItem(11).cast(IntegerType()))
result = result.withColumn('Family Film', split_col.getItem(12).cast(IntegerType()))
result = result.withColumn('Short Film', split_col.getItem(13).cast(IntegerType()))
result = result.withColumn('Romantic drama', split_col.getItem(14).cast(IntegerType()))
result = result.withColumn('Animation', split_col.getItem(15).cast(IntegerType()))
result = result.withColumn('Musical', split_col.getItem(16).cast(IntegerType()))
result = result.withColumn('Science Fiction', split_col.getItem(17).cast(IntegerType()))
result = result.withColumn('Mystery', split_col.getItem(18).cast(IntegerType()))
result = result.withColumn('Romantic Comedy', split_col.getItem(19).cast(IntegerType()))
result.show()



In [43]:
result = result.where("movie_id != 20580938")


In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer,Word2Vec
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
test_regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
test_tokenized = test_regexTokenizer.transform(test_df)
#tokenized.show()
# stop words
#add_stopwords = ["http","https","amp","rt","t","c","the"] 
test_stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
test_stop = test_stopwordsRemover.transform(test_tokenized)

# bag of words count
test_word2vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'filtered', outputCol = 'features')
test_model = test_word2vec.fit(test_stop)
test_result = test_model.transform(test_stop)
test_result.show()

In [44]:
from pyspark.ml.classification import LogisticRegression

lr1 = LogisticRegression(labelCol="Drama", featuresCol="features")
lrModel1 = lr1.fit(result)
output1 = lrModel1.transform(test_result)

lr2 = LogisticRegression(labelCol="Comedy", featuresCol="features")
lrModel2 = lr2.fit(result)
output2 = lrModel2.transform(test_result)

lr3 = LogisticRegression(labelCol="Romance Film", featuresCol="features")
lrModel3 = lr3.fit(result)
output3 = lrModel3.transform(test_result)

lr4 = LogisticRegression(labelCol="Thriller", featuresCol="features")
lrModel4 = lr4.fit(result)
output4 = lrModel4.transform(test_result)

lr5 = LogisticRegression(labelCol="Action", featuresCol="features")
lrModel5 = lr5.fit(result)
output5 = lrModel5.transform(test_result)

lr6 = LogisticRegression(labelCol="World Cinema", featuresCol="features")
lrModel6 = lr6.fit(result)
output6 = lrModel6.transform(test_result)

lr7 = LogisticRegression(labelCol="Crime Fiction", featuresCol="features")
lrModel7 = lr7.fit(result)
output7 = lrModel7.transform(test_result)

lr8 = LogisticRegression(labelCol="Horror", featuresCol="features")
lrModel8 = lr8.fit(result)
output8 = lrModel8.transform(test_result)

lr9 = LogisticRegression(labelCol="Black-and-White", featuresCol="features")
lrModel9 = lr9.fit(result)
output9 = lrModel9.transform(test_result)

lr10 = LogisticRegression(labelCol="Indie", featuresCol="features")
lrModel10 = lr10.fit(result)
output10 = lrModel10.transform(test_result)

lr11 = LogisticRegression(labelCol="Action/Adventure", featuresCol="features")
lrModel11 = lr11.fit(result)
output11 = lrModel11.transform(test_result)

lr12 = LogisticRegression(labelCol="Adventure", featuresCol="features")
lrModel12 = lr12.fit(result)
output12 = lrModel12.transform(test_result)

lr13 = LogisticRegression(labelCol="Family Film", featuresCol="features")
lrModel13 = lr13.fit(result)
output13 = lrModel13.transform(test_result)

lr14 = LogisticRegression(labelCol="Short Film", featuresCol="features")
lrModel14 = lr14.fit(result)
output14 = lrModel14.transform(test_result)

lr15 = LogisticRegression(labelCol="Romantic drama", featuresCol="features")
lrModel15 = lr15.fit(result)
output15 = lrModel15.transform(test_result)

lr16 = LogisticRegression(labelCol="Animation", featuresCol="features")
lrModel16 = lr16.fit(result)
output16 = lrModel16.transform(test_result)

lr17 = LogisticRegression(labelCol="Musical", featuresCol="features")
lrModel17 = lr17.fit(result)
output17 = lrModel17.transform(test_result)

lr18 = LogisticRegression(labelCol="Science Fiction", featuresCol="features")
lrModel18 = lr18.fit(result)
output18 = lrModel18.transform(test_result)

lr19 = LogisticRegression(labelCol="Mystery", featuresCol="features")
lrModel19 = lr19.fit(result)
output19 = lrModel19.transform(test_result)

lr20 = LogisticRegression(labelCol="Romantic Comedy", featuresCol="features")
lrModel20 = lr20.fit(result)
output20 = lrModel20.transform(test_result)



+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|          movie_name|                plot|               words|            filtered|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|[-0.0442120403582...|[1.05978177904513...|[0.74264884078618...|       0.0|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|[-0.0742260838540...|[0.76330854876330...|[0.68207162413300...|       0.0|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|[-0.0573536608455...|[1.11724446591884...|[0.75347723552209..

In [45]:
movie_list = output1.select('movie_id').collect()
movie_array = [int(row.movie_id) for row in movie_list]

#print(movie_array[:20])

mvv_list1 = output1.select('prediction').collect()
mvv_array1 = [int(row.prediction) for row in mvv_list1]
#print(mvv_array)

mvv_list2 = output2.select('prediction').collect()
mvv_array2 = [int(row.prediction) for row in mvv_list2]

mvv_list3 = output3.select('prediction').collect()
mvv_array3 = [int(row.prediction) for row in mvv_list3]

mvv_list4 = output4.select('prediction').collect()
mvv_array4 = [int(row.prediction) for row in mvv_list4]

mvv_list5 = output5.select('prediction').collect()
mvv_array5 = [int(row.prediction) for row in mvv_list5]

mvv_list6 = output6.select('prediction').collect()
mvv_array6 = [int(row.prediction) for row in mvv_list6]

mvv_list7 = output7.select('prediction').collect()
mvv_array7 = [int(row.prediction) for row in mvv_list7]

mvv_list8 = output8.select('prediction').collect()
mvv_array8 = [int(row.prediction) for row in mvv_list8]

mvv_list9 = output9.select('prediction').collect()
mvv_array9 = [int(row.prediction) for row in mvv_list9]

mvv_list10 = output10.select('prediction').collect()
mvv_array10 = [int(row.prediction) for row in mvv_list10]

mvv_list11 = output11.select('prediction').collect()
mvv_array11 = [int(row.prediction) for row in mvv_list11]
#print(mvv_array)

mvv_list12 = output12.select('prediction').collect()
mvv_array12 = [int(row.prediction) for row in mvv_list12]

mvv_list13 = output13.select('prediction').collect()
mvv_array13 = [int(row.prediction) for row in mvv_list13]

mvv_list14 = output14.select('prediction').collect()
mvv_array14 = [int(row.prediction) for row in mvv_list14]

mvv_list15 = output15.select('prediction').collect()
mvv_array15 = [int(row.prediction) for row in mvv_list15]

mvv_list16 = output16.select('prediction').collect()
mvv_array16 = [int(row.prediction) for row in mvv_list16]

mvv_list17 = output17.select('prediction').collect()
mvv_array17 = [int(row.prediction) for row in mvv_list17]

mvv_list18 = output18.select('prediction').collect()
mvv_array18 = [int(row.prediction) for row in mvv_list18]

mvv_list19 = output19.select('prediction').collect()
mvv_array19 = [int(row.prediction) for row in mvv_list19]

mvv_list20 = output20.select('prediction').collect()
mvv_array20 = [int(row.prediction) for row in mvv_list20]


string_list = []
for i in range(len(mvv_array1)):
    string = ''
    string = str(mvv_array1[i]) + ' ' + str(mvv_array2[i]) + ' ' + str(mvv_array3[i]) + ' ' + str(mvv_array4[i]) + ' ' + str(mvv_array5[i]) + ' ' + str(mvv_array6[i]) + ' ' + str(mvv_array7[i]) + ' ' + str(mvv_array8[i]) + ' ' + str(mvv_array9[i]) + ' ' + str(mvv_array10[i]) + ' ' + str(mvv_array11[i]) + ' ' + str(mvv_array12[i]) + ' ' + str(mvv_array13[i]) + ' ' + str(mvv_array14[i]) + ' ' + str(mvv_array15[i]) + ' ' + str(mvv_array16[i]) + ' ' + str(mvv_array17[i]) + ' ' + str(mvv_array18[i]) + ' ' + str(mvv_array19[i]) + ' ' + str(mvv_array20[i]) 
    string_list.append(string)



[1335380, 29062594, 9252321, 13455076, 24165951, 1925869, 10799612, 28238240, 17124781, 28207941, 19174305, 18392317, 34420857, 4039635, 8034072, 4016437, 1520023, 24589422, 35068740, 21132951]


In [ ]:
import pandas
import csv
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)
temp = sqlContext.createDataFrame(zip(movie_array,string_list),schema=['movie_id','predictions']) 
temp.toPandas().to_csv('part3.csv',index=False)